In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np

### Importing the Data

The data is comma seperated, and the raw data being read from absolulute directory /data/raw/

In [3]:
df = pd.read_csv("../data/raw/autoscout24-germany-dataset.csv")

display(df)

,mileage,make,model,fuel,gear,offerType,price,hp,year
0,235000,BMW,316,Diesel,Manual,Used,6800,116.0,2011
1,92800,Volkswagen,Golf,Gasoline,Manual,Used,6877,122.0,2011
2,149300,SEAT,Exeo,Gasoline,Manual,Used,6900,160.0,2011
3,96200,Renault,Megane,Gasoline,Manual,Used,6950,110.0,2011
4,156000,Peugeot,308,Gasoline,Manual,Used,6950,156.0,2011
...,...,...,...,...,...,...,...,...,...
46400,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46401,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46402,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46403,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021


### Cleaning Data

The data is 99.3% complete, but unfortunately there are 334 rows (out of 46405 rows) that contain incomplete information about the vehicle.
- Mileage, , contain complete information.
- 143 rows contain missing vehicle model, for these cases I am going to replace the blank space with "Unkown" since the data associated with the vehicle is still valuable even if the model is unkown.
- 182 rows contain missing transmission information

In [21]:
#display(df[df.isna().any(axis=1)])

df["model"] = df["model"].fillna("Unkown")

display(df[df["gear"].isna()])

display(df)

#display(df[df["model"] == "Vesta"])

,mileage,make,model,fuel,gear,offerType,price,hp,year
243,93850,Renault,Grand Scenic,Diesel,NaN,Used,7990,110.0,2012
347,52753,Lada,Vesta,Gasoline,NaN,Used,8000,106.0,2017
612,46850,Suzuki,Alto,Gasoline,NaN,Used,3999,68.0,2011
873,153500,Mitsubishi,ASX,Diesel,NaN,Used,10000,150.0,2014
1333,38000,Renault,ZOE,Electric,NaN,Used,8750,88.0,2016
...,...,...,...,...,...,...,...,...,...
44540,5000,Volvo,Unkown,Others,NaN,Used,2499,1.0,2020
44908,1490,Renault,Megane,Electric/Gasoline,NaN,Demonstration,21980,158.0,2020
45605,250,Fiat,New Panda,Gasoline,NaN,Demonstration,12990,69.0,2021
46326,45,BMW,iX3,Electric,NaN,Used,58000,286.0,2021


,mileage,make,model,fuel,gear,offerType,price,hp,year
0,235000,BMW,316,Diesel,Manual,Used,6800,116.0,2011
1,92800,Volkswagen,Golf,Gasoline,Manual,Used,6877,122.0,2011
2,149300,SEAT,Exeo,Gasoline,Manual,Used,6900,160.0,2011
3,96200,Renault,Megane,Gasoline,Manual,Used,6950,110.0,2011
4,156000,Peugeot,308,Gasoline,Manual,Used,6950,156.0,2011
...,...,...,...,...,...,...,...,...,...
46400,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46401,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46402,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
46403,99,Fiat,500,Electric/Gasoline,Manual,Pre-registered,12990,71.0,2021
